In [ ]:
#!pip install pandas==1.3.5 # for comaparability woth colab
#%pip install levenshtein
%pip freeze > session_info_sunflower.txt 

In [ ]:
import os 
import pickle 
import pandas as pd
import yaml
import sys
from datetime import date

import importlib
sys.path.append('../3-data-preprocessing')
import preprocessing_utils
importlib.reload(preprocessing_utils)

import traverse_and_parse as tp
importlib.reload(tp)

In [ ]:
# PATHS 
PATH_TO_PARSE = '../../data/QuantLet'
FILE_TYPES = ['m', 'py', 'r', 'R', 'M', 'ipynb']
TEST_PATH = '../data/Q_test'
OUTPUT_PATH = f"../../data/preprocessed/Quantlet/{TODAY}"

# CONSTANTS
IPY_CONVERT = False

# VARIABLE PARAMETERS
TODAY = '20231027'

if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

In [ ]:
if IPY_CONVERT:
    # q_name, folder_name, code_script, type_script, metainfo_file
    language, name = '', ''
    for i, (root, directories, files) in enumerate(os.walk(PATH_TO_PARSE)):
        
        
        for file in files:
            language = file.split('.')[-1]
            
            if (language == 'ipynb'):
                if not os.path.exists(f"{root}/{file.replace('ipynb', 'py')}"):
                    print(file)
                    os.popen(f"jupyter nbconvert --to python {root}/{file}")
            else:
                continue    

In [ ]:
output = tp.traverse_folder(PATH_TO_PARSE, FILE_TYPES)
repos = tp.prepare_repos_df(output['repos'])

In [ ]:
repos.type_script = repos.type_script.str.lower()
repos = repos[repos.type_script.isin(
            ['py', 'r', 'm', 'ipynb']
        )
    ]
repos_df = repos[~repos.folder_name.str.contains('.ipynb_checkpoints')]
repos_df.loc[repos_df.type_script== 'ipynb', 'type_script']='py'
repos_df = repos_df.reset_index(drop=True)
print(repos_df.shape)

In [ ]:
NAME = f'{OUTPUT_PATH}/Quantlets_{TODAY}'
with open(f'{NAME}.pkl', 'wb') as file:
    pickle.dump(repos_df, file)